## ETL USER


In [4]:
# Se cargan las librerias necesarias
import pandas as pd 
import numpy as np 
import json
from pandas import json_normalize
import ast
import gzip


Extraccion y exploracion del dataset reviews

In [5]:
# CArga del archivo cuando esta descomprimido
# Se leen los datos, se guardan en un dataframe y se visualiza la estructura que tiene
#reviews_ = []
#with open('australian_user_reviews.json', 'r', encoding='utf-8') as r:
#    for line in r:
#        reviews_.append(ast.literal_eval(line))

#reviews = pd.DataFrame(reviews_)
#reviews.head()


In [6]:
reviews_ = []

# Ruta al archivo comprimido que contiene los datos en formato Gzip
ruta_archivo_comprimido = 'user_reviews.json.gz'

# Abre el archivo comprimido en modo lectura
with gzip.open(ruta_archivo_comprimido, 'rt', encoding='utf-8') as archivo_gzip:
    # Lee el contenido del archivo Gzip línea por línea
    for line in archivo_gzip:
        # Usa ast.literal_eval para interpretar cada línea como un objeto Python
        review = ast.literal_eval(line)
        reviews_.append(review)

# Crea un DataFrame de Pandas con los datos
reviews = pd.DataFrame(reviews_)
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Se verifica el tipo de datos y la cantidad de datos de cada columna del dataframe

In [7]:
print(reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB
None


Se busca si hay filas duplicadas

In [8]:
duplicados = reviews.duplicated(subset='user_id', keep=False)

filas_duplicadas = reviews[duplicados]

print("La cantidad de filas duplicadas es:", len(filas_duplicadas)) 
filas_duplicadas.head()  

La cantidad de filas duplicadas es: 623


,user_id,user_url,reviews
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."
50,Rivtex,http://steamcommunity.com/id/Rivtex,"[{'funny': '', 'posted': 'Posted December 23, ..."
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,[]
119,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"[{'funny': '', 'posted': 'Posted August 29, 20..."
147,relesprit,http://steamcommunity.com/id/relesprit,"[{'funny': '', 'posted': 'Posted December 27, ..."


En el dataframe reviews  se revisa la estructura que tiene la columna reviews para poder tomar una decision con las filas duplicadas

In [9]:
reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

### Estructura de la informacion
* __user_id__: identificador único para el usuario.
* __user_url__: url del perfil del usuario en streamcommunity.
* __reviews__: contiene una lista de diccionarios. Para cada usuario se tiene uno o mas diccionarios. Cada diccionario contiene:
    * __funny__: indica si alguien puso emoticón de gracioso al review.
    * __posted__: fecha de posteo del review en formato Posted April 21, 2011.
    * __last_edited__: fecha de la última edición.
    * __item_id__: identificador único del juego.
    * __helpful__: estadística donde otros usuarios indican si fue útil la información.
    * __recommend__: booleano que indica si el usuario recomienda o no el juego.
    * __review__: comentarios sobre el juego.

Se escojen 2 user_id al azar para comprobar si realmente son filas duplicadas

In [10]:
# Se buscan dos user_id diferentes al azar
user_ids = filas_duplicadas['user_id'].sample(n=2, replace=False, random_state=42).tolist()

# Se Filtra el DataFrame para obtener las filas de los dos user_id seleccionados
filas = filas_duplicadas[filas_duplicadas['user_id'].isin(user_ids)]

# Imprimir las filas seleccionadas
print("Filas seleccionadas para los user_id:", user_ids)
filas.head()

Filas seleccionadas para los user_id: ['Justmatel', '76561198026239418']


,user_id,user_url,reviews
1279,Justmatel,http://steamcommunity.com/id/Justmatel,"[{'funny': '', 'posted': 'Posted April 8.', 'l..."
5506,Justmatel,http://steamcommunity.com/id/Justmatel,"[{'funny': '', 'posted': 'Posted April 8.', 'l..."
14947,76561198026239418,http://steamcommunity.com/profiles/76561198026...,"[{'funny': '2 people found this review funny',..."
14948,76561198026239418,http://steamcommunity.com/profiles/76561198026...,"[{'funny': '2 people found this review funny',..."


In [11]:
# Verificar e imprimir el primer review de la lista de diccionarios
for user_id, group in filas.groupby('user_id'):
    print(f"\nPara el user_id {user_id}:")
    for index, row in group.iterrows():
        first_review = row['reviews'][0]['review']
        print(f"Primer review: {first_review}")



Para el user_id 76561198026239418:
Primer review: good game 8/10▒▒▒▒▒▒▒▒█▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀█▒▒▒▒▒▒▒█░▒▒▒▒▒▒▒▓▒▒▓▒▒▒▒▒▒▒░█▒▒▒▒▒▒▒█░▒▒▓▒▒▒▒▒▒▒▒▒▄▄▒▓▒▒░█░▄▄▒▒▄▀▀▄▄█░▒▒▒▒▒▒▓▒▒▒▒█░░▀▄▄▄▄▄▀░░█▒▒█░░░░█░▒▒▒▒▒▒▒▒▒▒▒█░░░░░░░░░░░█▒▒▒▀▀▄▄█░▒▒▒▒▓▒▒▒▓▒█░░░█▒░░░░█▒░░█▒▒▒▒▒▒▒█░▒▓▒▒▒▒▓▒▒▒█░░░░░░░▀░░░░░█▒▒▒▒▒▄▄█░▒▒▒▓▒▒▒▒▒▒▒█░░█▄▄█▄▄█░░█▒▒▒▒█░░░█▄▄▄▄▄▄▄▄▄▄█░█▄▄▄▄▄▄▄▄▄█▒▒▒▒█▄▄█░░█▄▄█░░░░░░█▄▄█░░█▄▄█
Primer review: good game 8/10▒▒▒▒▒▒▒▒█▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀█▒▒▒▒▒▒▒█░▒▒▒▒▒▒▒▓▒▒▓▒▒▒▒▒▒▒░█▒▒▒▒▒▒▒█░▒▒▓▒▒▒▒▒▒▒▒▒▄▄▒▓▒▒░█░▄▄▒▒▄▀▀▄▄█░▒▒▒▒▒▒▓▒▒▒▒█░░▀▄▄▄▄▄▀░░█▒▒█░░░░█░▒▒▒▒▒▒▒▒▒▒▒█░░░░░░░░░░░█▒▒▒▀▀▄▄█░▒▒▒▒▓▒▒▒▓▒█░░░█▒░░░░█▒░░█▒▒▒▒▒▒▒█░▒▓▒▒▒▒▓▒▒▒█░░░░░░░▀░░░░░█▒▒▒▒▒▄▄█░▒▒▒▓▒▒▒▒▒▒▒█░░█▄▄█▄▄█░░█▒▒▒▒█░░░█▄▄▄▄▄▄▄▄▄▄█░█▄▄▄▄▄▄▄▄▄█▒▒▒▒█▄▄█░░█▄▄█░░░░░░█▄▄█░░█▄▄█

Para el user_id Justmatel:
Primer review: i have been playing since 07 and It's better than ever before!i dont play via steam so this doesn't represent my many hours of play and trade.
Primer review: i have been playing since 07 and It's better than ever before!i don

Mediante esta prueba se puede verificar que no solo estan duplicados los user_id sino tambien la demas informacion.

Teniendo en cuenta lo anterior se procede a borrar los duplicados.

In [12]:
reviews = reviews.drop_duplicates(subset='user_id', keep='first')
duplicados = reviews.duplicated(subset='user_id')
duplicados =duplicados.sum()
print("La cantidad de filas duplicadas es:", duplicados)

La cantidad de filas duplicadas es: 0


Desanidar la columna reviews del dataframe

In [13]:
# Desanidar la columna 'reviews'
reviews_des = pd.concat([reviews.drop(['reviews'], axis=1), 
                        json_normalize(reviews['reviews'])], axis=1)
reviews_des.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Las nuevas columnas que son ahora numeros se deben volver filas

In [14]:
reviews_des = pd.melt(reviews_des, id_vars=['user_id', 'user_url'], value_vars=list(range(9)), value_name='reviews')
reviews_des.head()


,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


Se toma un user_id y se observa como quedo

In [15]:
reviews_des[reviews_des['user_id'] == 'evcentric']

,user_id,user_url,variable,reviews
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
25801,evcentric,http://steamcommunity.com/id/evcentric,1,"{'funny': '', 'posted': 'Posted December 4, 20..."
51600,evcentric,http://steamcommunity.com/id/evcentric,2,"{'funny': '', 'posted': 'Posted November 3, 20..."
77399,evcentric,http://steamcommunity.com/id/evcentric,3,"{'funny': '', 'posted': 'Posted October 15, 20..."
103198,evcentric,http://steamcommunity.com/id/evcentric,4,"{'funny': '', 'posted': 'Posted October 15, 20..."
128997,evcentric,http://steamcommunity.com/id/evcentric,5,"{'funny': '', 'posted': 'Posted October 15, 20..."
154796,evcentric,http://steamcommunity.com/id/evcentric,6,None
180595,evcentric,http://steamcommunity.com/id/evcentric,7,None
206394,evcentric,http://steamcommunity.com/id/evcentric,8,None


Se comprueba la forma de los datos y se procede a borrar las filas que tengan None

In [16]:
reviews_des = reviews_des.dropna()
reviews_des[reviews_des['user_id'] == 'evcentric']

,user_id,user_url,variable,reviews
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
25801,evcentric,http://steamcommunity.com/id/evcentric,1,"{'funny': '', 'posted': 'Posted December 4, 20..."
51600,evcentric,http://steamcommunity.com/id/evcentric,2,"{'funny': '', 'posted': 'Posted November 3, 20..."
77399,evcentric,http://steamcommunity.com/id/evcentric,3,"{'funny': '', 'posted': 'Posted October 15, 20..."
103198,evcentric,http://steamcommunity.com/id/evcentric,4,"{'funny': '', 'posted': 'Posted October 15, 20..."
128997,evcentric,http://steamcommunity.com/id/evcentric,5,"{'funny': '', 'posted': 'Posted October 15, 20..."


Ahora del diccionario que esta en la columna reviews, cada clave se vuleve una columna nueva dataframe

In [17]:
# Se toman los diciconarios de la columna reviews y se vuelven series de pandas y despues se unen con dataframe 
expanded_reviews = reviews_des['reviews'].apply(pd.Series)
expanded_reviews = expanded_reviews.add_prefix('rev_')
reviews_des = pd.concat([reviews_des, expanded_reviews], axis=1).drop('reviews', axis=1)
reviews_des.head()

,user_id,user_url,variable,rev_funny,rev_posted,rev_last_edited,rev_item_id,rev_helpful,rev_recommend,rev_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,0,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,0,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,0,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se puede observar campos vacios, se procede a cambiarlos por None y se verifica si quedan aun campos vacios

In [18]:
reviews_des.replace('', None, inplace=True)
(reviews_des == '').sum()

user_id            0
user_url           0
variable           0
rev_funny          0
rev_posted         0
rev_last_edited    0
rev_item_id        0
rev_helpful        0
rev_recommend      0
rev_review         0
dtype: int64

In [19]:
reviews_des.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57397 entries, 0 to 231501
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          57397 non-null  object
 1   user_url         57397 non-null  object
 2   variable         57397 non-null  object
 3   rev_funny        7899 non-null   object
 4   rev_posted       57397 non-null  object
 5   rev_last_edited  5898 non-null   object
 6   rev_item_id      57397 non-null  object
 7   rev_helpful      57397 non-null  object
 8   rev_recommend    57397 non-null  bool  
 9   rev_review       57367 non-null  object
dtypes: bool(1), object(9)
memory usage: 4.4+ MB


Las columnas rev_funny y rev_last_edited tienen muy pocos datos, se toma la decision de borrarlas

In [20]:
reviews_des = reviews_des.drop(columns=['rev_funny', 'rev_last_edited'])

Se extrae de la columna rev_posted el año en que fue hecho

In [21]:
# Se extrae el año y se forma una nueva columna y se le da manejo a las filas que no se puede leer el año
reviews_des['año'] = reviews_des['rev_posted'].str.extract(r'(\d{4})')
reviews_des.loc[reviews_des['año'].isnull(), 'año'] = 'invalido'
reviews_des['año']

0             2011
1             2014
2         invalido
3             2013
4             2014
            ...   
231291        2014
231293        2014
231419        2015
231499        2015
231501    invalido
Name: año, Length: 57397, dtype: object

In [22]:
reviews_des[reviews_des['año'] == 'invalido']

,user_id,user_url,variable,rev_posted,rev_item_id,rev_helpful,rev_recommend,rev_review,año
2,evcentric,http://steamcommunity.com/id/evcentric,0,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,invalido
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,0,Posted May 20.,730,0 of 1 people (0%) found this review helpful,True,ZIKA DO BAILE,invalido
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,0,Posted July 24.,730,No ratings yet,True,BEST GAME IN THE BLOODY WORLD,invalido
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,0,Posted June 16.,252950,0 of 1 people (0%) found this review helpful,True,love it,invalido
10,76561198077246154,http://steamcommunity.com/profiles/76561198077...,0,Posted June 11.,440,No ratings yet,True,mt bom,invalido
...,...,...,...,...,...,...,...,...,...
223569,76561198040184950,http://steamcommunity.com/profiles/76561198040...,8,Posted April 12.,394690,No ratings yet,True,I cannot say much right now due to the game no...,invalido
226105,76561198046474248,http://steamcommunity.com/profiles/76561198046...,8,Posted March 28.,234140,No ratings yet,True,"Oh what a day .., What a lovely day to play th...",invalido
228109,dmitry_who,http://steamcommunity.com/id/dmitry_who,8,Posted May 17.,376210,10 of 28 people (36%) found this review helpful,True,░░░░░░░░░░░█▀▀░░█░░░░░░░░░░░▄▀▀▀▀░░░░░█▄▄░░░░░...,invalido
229231,76561198079507136,http://steamcommunity.com/profiles/76561198079...,8,Posted January 3.,730,No ratings yet,False,got VACed,invalido


Se elimina la columna rev_posted pues ya no tiene mas informacion relevante para el trabajo a realizar y ademas se borran los valores nulos de la columna rev_review	
pues no es mas del 5% de  los datos

In [23]:
reviews_des = reviews_des.drop('rev_posted', axis=1)
reviews_des = reviews_des.dropna(subset=['rev_review'])
reviews_des.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57367 entries, 0 to 231501
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        57367 non-null  object
 1   user_url       57367 non-null  object
 2   variable       57367 non-null  object
 3   rev_item_id    57367 non-null  object
 4   rev_helpful    57367 non-null  object
 5   rev_recommend  57367 non-null  bool  
 6   rev_review     57367 non-null  object
 7   año            57367 non-null  object
dtypes: bool(1), object(7)
memory usage: 3.6+ MB


Se guarda el dataset limpio

In [24]:
reviews_des.to_csv('review_clean.csv', index=False, encoding='utf-8')
print(f'Se guardó el archivo')

Se guardó el archivo
